<a href="https://colab.research.google.com/github/Bijayanee/CI_Projects/blob/main/Madaline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("priyanka841/heart-disease-prediction-uci")

print("Path to dataset files:", path)

100%|██████████| 3.40k/3.40k [00:00<00:00, 5.83MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/priyanka841/heart-disease-prediction-uci/versions/1


In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [6]:
df = pd.read_csv(path + "/heart.csv")
print(df.head())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  


In [7]:
X = df.drop('target', axis=1).values
y = df['target'].values

# Convert targets to bipolar (-1, 1) for MADALINE
y = np.where(y == 0, -1, 1)

# Normalize features to [-1, 1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [13]:
# Define MADALINE Model
class MADALINE(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MADALINE, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, 1)
        self.act = nn.Tanh()  # Bipolar activation

    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.act(self.output(x))
        return x

In [36]:
# Initialize model, loss, optimizer
model = MADALINE(input_size=X_train.shape[1], hidden_size=10)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# Train model
epochs = 200
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/200], Loss: 0.9653
Epoch [20/200], Loss: 0.9032
Epoch [30/200], Loss: 0.8500
Epoch [40/200], Loss: 0.8045
Epoch [50/200], Loss: 0.7655
Epoch [60/200], Loss: 0.7323
Epoch [70/200], Loss: 0.7042
Epoch [80/200], Loss: 0.6803
Epoch [90/200], Loss: 0.6601
Epoch [100/200], Loss: 0.6428
Epoch [110/200], Loss: 0.6280
Epoch [120/200], Loss: 0.6151
Epoch [130/200], Loss: 0.6038
Epoch [140/200], Loss: 0.5937
Epoch [150/200], Loss: 0.5848
Epoch [160/200], Loss: 0.5767
Epoch [170/200], Loss: 0.5693
Epoch [180/200], Loss: 0.5626
Epoch [190/200], Loss: 0.5564
Epoch [200/200], Loss: 0.5507


In [38]:
# Predictions & Accuracy
with torch.no_grad():
    predictions = model(X_test)
    predicted_classes = torch.where(predictions >= 0, 1, -1)
    accuracy = (predicted_classes == y_test).sum().item() / y_test.size(0)
    print(f"\nTest Accuracy: {accuracy*100:.2f}%\n")
    print("Sample Predictions (Actual vs Predicted):")
    for i in range(10):
        print(f"Actual: {int(y_test[i].item())}   Predicted: {int(predicted_classes[i].item())}")


Test Accuracy: 88.52%

Sample Predictions (Actual vs Predicted):
Actual: -1   Predicted: -1
Actual: -1   Predicted: 1
Actual: 1   Predicted: 1
Actual: -1   Predicted: -1
Actual: 1   Predicted: 1
Actual: 1   Predicted: 1
Actual: 1   Predicted: 1
Actual: -1   Predicted: -1
Actual: -1   Predicted: -1
Actual: 1   Predicted: 1
